In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 


pybullet build time: Jan 29 2025 23:19:57


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3050 Ti Laptop GPU/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.230.02
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.230.02
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3050 Ti Laptop GPU/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation


In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the elapsed time per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")



ven = NVIDIA Corporation


In [3]:
# Load the robot
car_start_pos = [0, 0, 0.1]  # Set the initial position (x, y, z)
car_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set the initial orientation (roll, pitch, yaw)
car_id = pybullet.loadURDF("../urdf/mobile_robot_with_arm.urdf", car_start_pos, car_start_orientation)

In [4]:
# Set initial arm joint positions
# Function to get joint index by name (will be defined again in later cell, but needed here)
def get_joint_index_by_name(robot_id, joint_name):
    """Get the joint index by searching through all joints."""
    num_joints = pybullet.getNumJoints(robot_id)
    for i in range(num_joints):
        joint_info = pybullet.getJointInfo(robot_id, i)
        joint_name_found = joint_info[1].decode('utf-8')
        if joint_name_found == joint_name:
            return i
    return None

# Set initial joint positions
# shoulder_lift_joint should be at -1.257 radians
initial_joint_positions = {
    'shoulder_pan_joint': 0.0,
    'shoulder_lift_joint': -1.257,  # Initial position as requested
    'elbow_joint': 0.0,
    'wrist_1_joint': 0.0,
    'wrist_2_joint': 0.0,
    'wrist_3_joint': 0.0
}

# Apply initial joint positions
for joint_name, joint_angle in initial_joint_positions.items():
    joint_idx = get_joint_index_by_name(car_id, joint_name)
    if joint_idx is not None:
        pybullet.resetJointState(car_id, joint_idx, joint_angle)
        print(f"Set initial position for {joint_name}: {joint_angle:.3f} rad")
    else:
        print(f"Warning: Could not find {joint_name} for initial positioning")

# Step simulation a few times to let the arm settle
for _ in range(10):
    pybullet.stepSimulation()


Set initial position for shoulder_pan_joint: 0.000 rad
Set initial position for shoulder_lift_joint: -1.257 rad
Set initial position for elbow_joint: 0.000 rad
Set initial position for wrist_1_joint: 0.000 rad
Set initial position for wrist_2_joint: 0.000 rad
Set initial position for wrist_3_joint: 0.000 rad


In [5]:
import random
import numpy as np

# Add multiple boxes at random positions in front of the camera
# Camera is mounted on the robot base, pointing forward (in +Y direction)
# Place boxes at various distances and positions in front of the camera

num_boxes = 5  # Number of boxes to add
box_ids = []  # Store box IDs

# Box size from URDF is 1x1x1 meters
box_size = 0.05

# Camera position relative to base: approximately [0.12, 0, 0.08] from base_link
# Base is at [0, 0, 0.1], so camera is roughly at [0.12, 0, 0.18] in world coordinates
# Place boxes in front of camera (positive Y direction) at distances between 1-5 meters

for i in range(num_boxes):
    # Random distance from camera (1 to 5 meters in front)
    distance = random.uniform(1.0, 5.0)
    
    # Random lateral position (-2 to 2 meters left/right)
    lateral_offset = random.uniform(-2.0, 2.0)
    
    # Random height offset (-0.5 to 1.5 meters relative to camera)
    height_offset = 0
    
    # Calculate box position
    # Camera is at base position + offset, pointing in +Y direction
    box_x = 0.12 + lateral_offset  # Lateral position
    box_y = distance  # Distance in front of camera
    # Box size is 0.6m, so half height is 0.3m. Place box at 0.3m so bottom is at ground level (z=0)
    box_z = 0.3 + height_offset  # Height relative to ground (0.3m = half box height to sit on ground)
    
    # Random orientation (optional - can be set to [0,0,0] for no rotation)
    box_roll = random.uniform(-0.2, 0.2)
    box_pitch = random.uniform(-0.2, 0.2)
    box_yaw = random.uniform(0, 2 * np.pi)
    box_orientation = pybullet.getQuaternionFromEuler([0, 0, box_yaw])
    
    # Load the box (useFixedBase=False so boxes can fall and move due to physics)
    box_id = pybullet.loadURDF("../urdf/simple_box.urdf", 
                               [box_y, box_x, box_z], 
                               box_orientation,
                               useFixedBase=False)
    box_ids.append(box_id)
    
    print(f"Box {i+1} loaded at position ({box_x:.2f}, {box_y:.2f}, {box_z:.2f}), distance: {distance:.2f}m")

print(f"\nTotal {num_boxes} boxes loaded in front of the camera.")


Box 1 loaded at position (2.07, 2.73, 0.30), distance: 2.73m
Box 2 loaded at position (1.00, 1.54, 0.30), distance: 1.54m
Box 3 loaded at position (-0.44, 1.37, 0.30), distance: 1.37m
Box 4 loaded at position (0.78, 2.53, 0.30), distance: 2.53m
Box 5 loaded at position (1.55, 4.49, 0.30), distance: 4.49m

Total 5 boxes loaded in front of the camera.


In [6]:
import cv2
import numpy as np
import time

# Camera settings (similar to AR marker detection notebook)
fov = 60  # Field of view in degrees
image_width = 640
image_height = 480
aspect = image_width / image_height
near = 0.05  # Near clipping plane
far = 10.0   # Far clipping plane

# Compute projection matrix
projection_matrix = pybullet.computeProjectionMatrixFOV(fov, aspect, near, far)

# Set camera view for GUI
camera_distance = 2.0
camera_yaw = 45.0
camera_pitch = -30.0
camera_target_position = [0.0, 0.0, 0.5]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)


In [7]:
# Function to get link index by name
def get_link_index_by_name(robot_id, link_name):
    """
    Get the link index by searching through all joints.
    In PyBullet, child link indices are the same as joint indices.
    """
    # Search through all joints
    num_joints = pybullet.getNumJoints(robot_id)
    for i in range(num_joints):
        joint_info = pybullet.getJointInfo(robot_id, i)
        child_link_name = joint_info[12].decode('utf-8')
        if child_link_name == link_name:
            # Return the child link index (same as joint index for child links)
            return i
    
    # If not found, return None
    return None

# Get camera link indices
rgbd_camera_link_idx = get_link_index_by_name(car_id, "rgbd_camera_link")
rgbd_camera_target_link_idx = get_link_index_by_name(car_id, "rgbd_camera_target_vertual_link")

print(f"RGB-D Camera Link Index: {rgbd_camera_link_idx}")
print(f"RGB-D Camera Target Link Index: {rgbd_camera_target_link_idx}")

# If we can't find by name, we'll use a fallback method
# Actually, in PyBullet, link indices for fixed joints are the same as joint indices
# Let's try to get link state directly
if rgbd_camera_link_idx is None:
    # Alternative: search through all links
    # For now, let's assume the links exist and try to access them
    # We'll handle errors in the main loop
    print("Warning: Could not find camera link by name. Will try to use joint indices.")


RGB-D Camera Link Index: 8
RGB-D Camera Target Link Index: 9


In [8]:
# Function to get 2D lidar point cloud
def get_2d_lidar_pointcloud(robot_id, lidar_link_idx, num_rays=360, max_range=10.0):
    """
    Get 2D point cloud from lidar using ray casting.
    
    Parameters:
    -----------
    robot_id : int
        PyBullet body ID of the robot
    lidar_link_idx : int
        Link index of the lidar
    num_rays : int
        Number of rays to cast (360 for full circle)
    max_range : float
        Maximum range of the lidar in meters
    
    Returns:
    --------
    points : numpy.ndarray
        2D array of shape (N, 2) containing [x, y] coordinates of points
    """
    # Get lidar position and orientation
    lidar_state = pybullet.getLinkState(robot_id, lidar_link_idx)
    lidar_position = np.array(lidar_state[0])  # [x, y, z]
    
    # Get base orientation for coordinate transformation
    base_state = pybullet.getBasePositionAndOrientation(robot_id)
    base_orientation = base_state[1]
    
    # Convert quaternion to rotation matrix
    rotation_matrix = np.array(pybullet.getMatrixFromQuaternion(base_orientation)).reshape(3, 3)
    
    points = []
    angles = np.linspace(0, 2 * np.pi, num_rays, endpoint=False)
    
    # Prepare ray starts and ends for batch processing
    ray_starts = []
    ray_ends = []
    ray_directions_world = []
    
    for angle in angles:
        # Calculate ray direction in lidar's local frame (horizontal plane, Z=0)
        # Lidar scans in a circle: 0° is forward (+X in local frame), rotating counterclockwise
        ray_direction_local = np.array([np.cos(angle), np.sin(angle), 0])
        
        # Rotate ray direction to world coordinates
        ray_direction_world = rotation_matrix @ ray_direction_local
        ray_directions_world.append(ray_direction_world)
        
        # Calculate ray end position
        ray_start = lidar_position
        ray_end = lidar_position + ray_direction_world * max_range
        
        ray_starts.append(ray_start)
        ray_ends.append(ray_end)
    
    # Cast all rays at once using rayTestBatch (more efficient)
    ray_results = pybullet.rayTestBatch(ray_starts, ray_ends)
    
    # Process results
    for i, angle in enumerate(angles):
        ray_result = ray_results[i]
        hit_object_id = ray_result[0]
        hit_fraction = ray_result[2]
        hit_position = ray_result[3]
        ray_direction_world = ray_directions_world[i]
        
        # Check if we hit something and it's not part of the robot
        if hit_object_id != -1 and hit_object_id != robot_id:
            # Hit an external object
            point_2d = np.array([hit_position[0] - lidar_position[0], 
                                hit_position[1] - lidar_position[1]])
            points.append(point_2d)
        elif hit_object_id == -1:
            # No hit, use max range point
            point_2d = np.array([ray_direction_world[0] * max_range,
                                ray_direction_world[1] * max_range])
            points.append(point_2d)
        else:
            # Hit the robot itself, try to extend the ray beyond the robot
            # Start the ray slightly further out to avoid self-collision
            min_distance = 0.4  # Minimum distance to avoid robot body (meters)
            ray_start_extended = lidar_position + ray_direction_world * min_distance
            ray_end_extended = lidar_position + ray_direction_world * max_range
            
            ray_result_extended = pybullet.rayTest(ray_start_extended, ray_end_extended)
            
            if ray_result_extended[0][0] != -1 and ray_result_extended[0][0] != robot_id:
                # Hit something beyond the robot
                hit_position_extended = ray_result_extended[0][3]
                point_2d = np.array([hit_position_extended[0] - lidar_position[0], 
                                    hit_position_extended[1] - lidar_position[1]])
                points.append(point_2d)
            else:
                # No hit beyond robot, use max range
                point_2d = np.array([ray_direction_world[0] * max_range,
                                    ray_direction_world[1] * max_range])
                points.append(point_2d)
    
    return np.array(points)

# Get lidar link index
lidar_link_idx = get_link_index_by_name(car_id, "lidar_link")
print(f"Lidar Link Index: {lidar_link_idx}")

# Lidar parameters
lidar_num_rays = 360  # Number of rays (360 for full circle)
lidar_max_range = 10.0  # Maximum range in meters


Lidar Link Index: 7


In [9]:
# Matplotlib setup is now integrated into the camera display cell
# This cell is kept for reference but the actual setup happens in the display loop
import matplotlib.pyplot as plt


In [10]:
# This cell has been merged into the camera display cell below
# Lidar visualization is now integrated with camera image display


In [ ]:
# Function to get joint index by name
def get_joint_index_by_name(robot_id, joint_name):
    """
    Get the joint index by searching through all joints.
    """
    num_joints = pybullet.getNumJoints(robot_id)
    for i in range(num_joints):
        joint_info = pybullet.getJointInfo(robot_id, i)
        joint_name_found = joint_info[1].decode('utf-8')
        if joint_name_found == joint_name:
            return i
    return None

# Get joint indices for the UR5 arm
arm_joint_indices = {}
# UR5 joint names
arm_joint_names = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 
                   'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']

for joint_name in arm_joint_names:
    joint_idx = get_joint_index_by_name(car_id, joint_name)
    arm_joint_indices[joint_name] = joint_idx
    if joint_idx is not None:
        print(f"{joint_name} index: {joint_idx}")
    else:
        print(f"Warning: {joint_name} not found!")

# Gripper joint names (both need to be controlled since PyBullet doesn't support mimic joints)
gripper_right_joint_name = 'robotiq_2f_85_right_driver_joint'
gripper_left_joint_name = 'robotiq_2f_85_left_driver_joint'

# Get gripper joint indices (both right and left)
gripper_right_joint_idx = get_joint_index_by_name(car_id, gripper_right_joint_name)
gripper_left_joint_idx = get_joint_index_by_name(car_id, gripper_left_joint_name)

if gripper_right_joint_idx is not None:
    print(f"{gripper_right_joint_name} index: {gripper_right_joint_idx}")
else:
    print(f"Warning: {gripper_right_joint_name} not found!")

if gripper_left_joint_idx is not None:
    print(f"{gripper_left_joint_name} index: {gripper_left_joint_idx}")
else:
    print(f"Warning: {gripper_left_joint_name} not found!")

# UR5 Joint limits (different for each joint)
joint_limits = {
    'shoulder_pan_joint': (-6.28318530718, 6.28318530718),  # ±360°
    'shoulder_lift_joint': (-6.28318530718, 6.28318530718),  # ±360°
    'elbow_joint': (-3.14159265359, 3.14159265359),  # ±180°
    'wrist_1_joint': (-6.28318530718, 6.28318530718),  # ±360°
    'wrist_2_joint': (-6.28318530718, 6.28318530718),  # ±360°
    'wrist_3_joint': (-6.28318530718, 6.28318530718)   # ±360°
}

# Gripper limits (0.0 = closed, 0.834 = fully open)
gripper_limits = (0.0, 0.834)

# UR5 Joint effort limits (maximum force/torque for each joint)
joint_efforts = {
    'shoulder_pan_joint': 150.0,
    'shoulder_lift_joint': 150.0,
    'elbow_joint': 150.0,
    'wrist_1_joint': 28.0,
    'wrist_2_joint': 28.0,
    'wrist_3_joint': 28.0
}

gripper_effort = 60.0  # Maximum force for gripper

# Set default joint positions (shoulder_lift_joint starts at -1.257)
joint_defaults = {
    'shoulder_pan_joint': 0.0,
    'shoulder_lift_joint': -1.257,  # Initial position as requested
    'elbow_joint': 0.0,
    'wrist_1_joint': 0.0,
    'wrist_2_joint': 0.0,
    'wrist_3_joint': 0.0
}

gripper_default = 0.0  # Start closed

# Add sliders for each joint in PyBullet GUI
# These will appear in the right panel of the PyBullet window
slider_ids = {}
for joint_name in arm_joint_names:
    joint_min, joint_max = joint_limits[joint_name]
    joint_default = joint_defaults.get(joint_name, 0.0)  # Get default for this joint
    slider_id = pybullet.addUserDebugParameter(
        joint_name,  # Parameter name
        joint_min,   # Minimum value (radians)
        joint_max,   # Maximum value (radians)
        joint_default  # Default value
    )
    slider_ids[joint_name] = slider_id
    print(f"Added slider for {joint_name} (slider ID: {slider_id}, range: [{joint_min:.2f}, {joint_max:.2f}] rad, default: {joint_default:.3f} rad)")

# Add slider for gripper (single slider for opening/closing)
gripper_slider_id = pybullet.addUserDebugParameter(
    "Gripper Open/Close",  # Parameter name
    gripper_limits[0],     # Minimum value (0.0 = closed)
    gripper_limits[1],     # Maximum value (0.834 = fully open)
    gripper_default        # Default value (closed)
)
print(f"Added slider for Gripper (slider ID: {gripper_slider_id}, range: [{gripper_limits[0]:.2f}, {gripper_limits[1]:.2f}] rad)")

# Get wheel joint indices for mobile base control
wheel_joint_names = [
    'left_wheel_joint',
    'right_wheel_joint',
    'front_left_wheel_joint',
    'front_right_wheel_joint'
]

wheel_joint_indices = {}
for wheel_name in wheel_joint_names:
    wheel_idx = get_joint_index_by_name(car_id, wheel_name)
    wheel_joint_indices[wheel_name] = wheel_idx
    if wheel_idx is not None:
        print(f"{wheel_name} index: {wheel_idx}")
    else:
        print(f"Warning: {wheel_name} not found!")

# Add slider for forward/backward movement
# Positive = forward, negative = backward
# All wheels move in the same direction with same angular velocity
# Range: -100 to 100 rad/s (with 0.08m wheel radius, this gives ~8 m/s linear velocity)
forward_backward_slider_id = pybullet.addUserDebugParameter(
    "Forward/Backward",  # Parameter name
    -100.0,             # Minimum value (rad/s, backward)
    100.0,              # Maximum value (rad/s, forward)
    0.0                 # Default value (stopped)
)
print(f"Added slider for Forward/Backward (slider ID: {forward_backward_slider_id}, range: [-100.0, 100.0] rad/s)")

# Add slider for rotation
# Positive = rotate counterclockwise (left wheels forward, right wheels backward)
# Negative = rotate clockwise (left wheels backward, right wheels forward)
# Range: -100 to 100 rad/s for strong rotation capability
rotation_slider_id = pybullet.addUserDebugParameter(
    "Rotate Left/Right",  # Parameter name
    -100.0,               # Minimum value (rad/s, clockwise)
    100.0,                # Maximum value (rad/s, counterclockwise)
    0.0                   # Default value (no rotation)
)
print(f"Added slider for Rotate Left/Right (slider ID: {rotation_slider_id}, range: [-100.0, 100.0] rad/s)")

# Maximum torque for wheel motors (significantly increased for better control and faster acceleration)
wheel_max_torque = 200.0  # N⋅m


shoulder_pan_joint index: 12
shoulder_lift_joint index: 13
elbow_joint index: 14
wrist_1_joint index: 15
wrist_2_joint index: 16
wrist_3_joint index: 17
robotiq_2f_85_right_driver_joint index: 20
robotiq_2f_85_left_driver_joint index: 25
Added slider for shoulder_pan_joint (slider ID: 0, range: [-6.28, 6.28] rad, default: 0.000 rad)
Added slider for shoulder_lift_joint (slider ID: 1, range: [-6.28, 6.28] rad, default: -1.257 rad)
Added slider for elbow_joint (slider ID: 2, range: [-3.14, 3.14] rad, default: 0.000 rad)
Added slider for wrist_1_joint (slider ID: 3, range: [-6.28, 6.28] rad, default: 0.000 rad)
Added slider for wrist_2_joint (slider ID: 4, range: [-6.28, 6.28] rad, default: 0.000 rad)
Added slider for wrist_3_joint (slider ID: 5, range: [-6.28, 6.28] rad, default: 0.000 rad)
Added slider for Gripper (slider ID: 6, range: [0.00, 0.83] rad)
left_wheel_joint index: 0
right_wheel_joint index: 1
front_left_wheel_joint index: 2
front_right_wheel_joint index: 3
Added slider for 

: 

In [ ]:
# Main loop to capture and display camera images and 2D LIDAR point cloud
# Press 'q' to quit the display windows

# Create named windows for display
cv2.namedWindow('RGB Image', cv2.WINDOW_NORMAL)
cv2.namedWindow('Depth Image', cv2.WINDOW_NORMAL)
cv2.namedWindow('Segmented Image', cv2.WINDOW_NORMAL)

# Set up parameters for 2D LIDAR point cloud visualization using OpenCV
if 'lidar_max_range' not in globals():
    lidar_max_range = 10.0

# Create OpenCV window for LIDAR visualization
cv2.namedWindow('2D Lidar Point Cloud', cv2.WINDOW_NORMAL)

# LIDAR visualization image size
lidar_img_size = 800  # Square image in pixels
lidar_img_center = lidar_img_size // 2
lidar_pixels_per_meter = lidar_img_size / (2 * lidar_max_range)  # Scale factor

try:
    frame_count = 0
    while True:
        # Read slider values and control arm joints
        for joint_name in arm_joint_names:
            if joint_name in slider_ids and joint_name in arm_joint_indices:
                slider_id = slider_ids[joint_name]
                joint_idx = arm_joint_indices[joint_name]
                if joint_idx is not None:
                    # Read slider value
                    joint_angle = pybullet.readUserDebugParameter(slider_id)
                    # Get joint effort limit (force/torque)
                    joint_effort = joint_efforts.get(joint_name, 150.0)
                    # Set joint position (control mode: position control)
                    pybullet.setJointMotorControl2(
                        car_id,
                        joint_idx,
                        pybullet.POSITION_CONTROL,
                        targetPosition=joint_angle,
                        force=joint_effort  # Maximum force/torque for this joint
                    )
        
        # Control gripper (both right and left driver joints)
        # Read gripper slider value once
        gripper_angle = pybullet.readUserDebugParameter(gripper_slider_id)
        
        # Control right driver joint
        if gripper_right_joint_idx is not None:
            pybullet.setJointMotorControl2(
                car_id,
                gripper_right_joint_idx,
                pybullet.POSITION_CONTROL,
                targetPosition=gripper_angle,
                force=gripper_effort
            )
        
        # Control left driver joint (same angle as right, since they should move together)
        if gripper_left_joint_idx is not None:
            pybullet.setJointMotorControl2(
                car_id,
                gripper_left_joint_idx,
                pybullet.POSITION_CONTROL,
                targetPosition=gripper_angle,
                force=gripper_effort
            )
        
        # Control mobile base wheels
        # Read forward/backward and rotation slider values
        forward_backward_velocity = pybullet.readUserDebugParameter(forward_backward_slider_id)
        rotation_velocity = pybullet.readUserDebugParameter(rotation_slider_id)
        
        # Calculate wheel velocities
        # Forward/backward: all wheels move in same direction
        # Rotation: left wheels and right wheels move in opposite directions
        left_wheel_velocity = forward_backward_velocity + rotation_velocity
        right_wheel_velocity = forward_backward_velocity - rotation_velocity
        
        # Apply velocities to all wheels using VELOCITY_CONTROL
        # Left wheels (left_wheel_joint, front_left_wheel_joint)
        if 'left_wheel_joint' in wheel_joint_indices and wheel_joint_indices['left_wheel_joint'] is not None:
            pybullet.setJointMotorControl2(
                car_id,
                wheel_joint_indices['left_wheel_joint'],
                pybullet.VELOCITY_CONTROL,
                targetVelocity=left_wheel_velocity,
                force=wheel_max_torque
            )
        
        if 'front_left_wheel_joint' in wheel_joint_indices and wheel_joint_indices['front_left_wheel_joint'] is not None:
            pybullet.setJointMotorControl2(
                car_id,
                wheel_joint_indices['front_left_wheel_joint'],
                pybullet.VELOCITY_CONTROL,
                targetVelocity=left_wheel_velocity,
                force=wheel_max_torque
            )
        
        # Right wheels (right_wheel_joint, front_right_wheel_joint)
        if 'right_wheel_joint' in wheel_joint_indices and wheel_joint_indices['right_wheel_joint'] is not None:
            pybullet.setJointMotorControl2(
                car_id,
                wheel_joint_indices['right_wheel_joint'],
                pybullet.VELOCITY_CONTROL,
                targetVelocity=right_wheel_velocity,
                force=wheel_max_torque
            )
        
        if 'front_right_wheel_joint' in wheel_joint_indices and wheel_joint_indices['front_right_wheel_joint'] is not None:
            pybullet.setJointMotorControl2(
                car_id,
                wheel_joint_indices['front_right_wheel_joint'],
                pybullet.VELOCITY_CONTROL,
                targetVelocity=right_wheel_velocity,
                force=wheel_max_torque
            )
        
        # Step simulation
        pybullet.stepSimulation()
        
        # Try to get camera link positions
        # First, let's find the correct link indices by checking all joints
        if rgbd_camera_link_idx is None or rgbd_camera_target_link_idx is None:
            # Search for camera links
            num_joints = pybullet.getNumJoints(car_id)
            for i in range(num_joints):
                joint_info = pybullet.getJointInfo(car_id, i)
                link_name = joint_info[12].decode('utf-8')
                if "rgbd_camera_link" in link_name and rgbd_camera_link_idx is None:
                    rgbd_camera_link_idx = i
                if "rgbd_camera_target" in link_name and rgbd_camera_target_link_idx is None:
                    rgbd_camera_target_link_idx = i
        
        # Get camera link positions and orientations
        try:
            if rgbd_camera_link_idx is not None:
                camera_link_state = pybullet.getLinkState(car_id, rgbd_camera_link_idx)
                camera_link_position = camera_link_state[0]  # World position
                camera_link_orientation = camera_link_state[1]  # World orientation (quaternion)
            else:
                # Fallback: use base link position + offset
                base_state = pybullet.getBasePositionAndOrientation(car_id)
                camera_link_position = [base_state[0][0] + 0.12, base_state[0][1], base_state[0][2] + 0.08]
                camera_link_orientation = base_state[1]
            
            if rgbd_camera_target_link_idx is not None:
                camera_target_state = pybullet.getLinkState(car_id, rgbd_camera_target_link_idx)
                camera_target_position = camera_target_state[0]
            else:
                # Fallback: position camera target 0.2m in front of camera
                # Convert quaternion to rotation matrix to get forward direction
                # For simplicity, assume camera points in +Y direction (forward)
                camera_target_position = [camera_link_position[0] + 0.2, 
                                         camera_link_position[1], 
                                         camera_link_position[2]]
        except:
            # If link access fails, use base position with offset
            base_state = pybullet.getBasePositionAndOrientation(car_id)
            camera_link_position = [base_state[0][0] + 0.12, base_state[0][1], base_state[0][2] + 0.08]
            camera_target_position = [camera_link_position[0] + 0.2, 
                                     camera_link_position[1], 
                                     camera_link_position[2]]
        
        # Compute view matrix
        view_matrix = pybullet.computeViewMatrix(
            cameraEyePosition=camera_link_position,
            cameraTargetPosition=camera_target_position,
            cameraUpVector=[0, 0, 1]  # Z-axis is up
        )
        
        # Get camera image (returns width, height, rgbImg, depthImg, segImg)
        width, height, rgb_img, depth_img, seg_img = pybullet.getCameraImage(
            width=image_width,
            height=image_height,
            viewMatrix=view_matrix,
            projectionMatrix=projection_matrix,
            renderer=pybullet.ER_BULLET_HARDWARE_OPENGL
        )
        
        # Process RGB image (already in correct format)
        rgb_display = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
        
        # Process depth image
        # Depth image is in meters, normalize for display (0-255)
        depth_array = np.array(depth_img)
        # Normalize depth to 0-255 range for visualization
        depth_normalized = cv2.normalize(depth_array, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        depth_colormap = cv2.applyColorMap(depth_normalized, cv2.COLORMAP_JET)
        
        # Process segmented image
        # Segmentation image contains object IDs, convert to color image
        seg_array = np.array(seg_img, dtype=np.int32)  # Object IDs can be signed integers
        # Reshape to image dimensions (height, width)
        seg_array = seg_array.reshape((height, width))
        
        # Get unique object IDs (excluding -1 which is typically "no object")
        unique_ids = np.unique(seg_array)
        unique_ids = unique_ids[unique_ids >= 0]  # Remove negative IDs (background/no object)
        
        # Create a color mapping for each object ID
        seg_colormap = np.zeros((height, width, 3), dtype=np.uint8)
        
        if len(unique_ids) > 0:
            # Generate distinct colors for each object ID
            # Use a hash-based approach to get consistent colors for each ID
            for obj_id in unique_ids:
                mask = seg_array == obj_id
                # Generate a color based on object ID using HSV color space
                # Use object ID to generate hue (0-179 for OpenCV HSV)
                hue = int((obj_id * 179) % 180)  # Wrap around HSV hue range
                # Create a full saturation, full value color
                color_hsv = np.uint8([[[hue, 255, 255]]])
                color_bgr = cv2.cvtColor(color_hsv, cv2.COLOR_HSV2BGR)[0][0]
                seg_colormap[mask] = color_bgr
        
        # For background/no object pixels (ID < 0 or not in unique_ids), use black
        background_mask = ~np.isin(seg_array, unique_ids)
        seg_colormap[background_mask] = [0, 0, 0]  # Black for background
        
        # Add bounding boxes for each detected object
        if len(unique_ids) > 0:
            for obj_id in unique_ids:
                mask = seg_array == obj_id
                # Find bounding box coordinates
                coords = np.column_stack(np.where(mask))
                if len(coords) > 0:
                    # Get min/max coordinates (note: y is first, x is second in np.where)
                    y_min, x_min = coords.min(axis=0)
                    y_max, x_max = coords.max(axis=0)
                    
                    # Get color for this object ID (same as used for the mask)
                    hue = int((obj_id * 179) % 180)
                    color_hsv = np.uint8([[[hue, 255, 255]]])
                    color_bgr = cv2.cvtColor(color_hsv, cv2.COLOR_HSV2BGR)[0][0]
                    color_tuple = tuple(map(int, color_bgr))
                    
                    # Draw bounding box (thick line for visibility)
                    cv2.rectangle(seg_colormap, (x_min, y_min), (x_max, y_max), 
                                 color_tuple, 2)
                    
                    # Add label with object ID
                    label = f"ID:{obj_id}"
                    label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                    # Draw background rectangle for text (slightly larger for padding)
                    cv2.rectangle(seg_colormap, (x_min, y_min - label_size[1] - 4), 
                                 (x_min + label_size[0] + 4, y_min), color_tuple, -1)
                    # Draw text in white for contrast
                    cv2.putText(seg_colormap, label, (x_min + 2, y_min - 2), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Display camera images
        cv2.imshow('RGB Image', rgb_display)
        cv2.imshow('Depth Image', depth_colormap)
        cv2.imshow('Segmented Image', seg_colormap)
        
        # Get and display 2D LIDAR point cloud
        if lidar_link_idx is not None:
            pointcloud = get_2d_lidar_pointcloud(car_id, lidar_link_idx, 
                                                num_rays=lidar_num_rays, 
                                                max_range=lidar_max_range)
            
            # Create image for LIDAR visualization
            lidar_img = np.ones((lidar_img_size, lidar_img_size, 3), dtype=np.uint8) * 255  # White background
            
            # Draw grid
            grid_spacing = 2.0  # meters
            for i in range(-int(lidar_max_range), int(lidar_max_range) + 1, int(grid_spacing)):
                x_pixel = int(lidar_img_center + i * lidar_pixels_per_meter)
                y_pixel = int(lidar_img_center + i * lidar_pixels_per_meter)
                if 0 <= x_pixel < lidar_img_size:
                    cv2.line(lidar_img, (x_pixel, 0), (x_pixel, lidar_img_size), (200, 200, 200), 1)
                if 0 <= y_pixel < lidar_img_size:
                    cv2.line(lidar_img, (0, y_pixel), (lidar_img_size, y_pixel), (200, 200, 200), 1)
            
            # Draw axes
            cv2.line(lidar_img, (lidar_img_center, 0), (lidar_img_center, lidar_img_size), (150, 150, 150), 2)
            cv2.line(lidar_img, (0, lidar_img_center), (lidar_img_size, lidar_img_center), (150, 150, 150), 2)
            
            # Draw LIDAR points
            for point in pointcloud:
                # Convert from meters to pixels
                # Note: OpenCV uses (x, y) where y increases downward, so we need to flip Y
                x_pixel = int(lidar_img_center + point[0] * lidar_pixels_per_meter)
                y_pixel = int(lidar_img_center - point[1] * lidar_pixels_per_meter)  # Flip Y axis
                
                # Draw point if within image bounds
                if 0 <= x_pixel < lidar_img_size and 0 <= y_pixel < lidar_img_size:
                    cv2.circle(lidar_img, (x_pixel, y_pixel), 2, (0, 0, 255), -1)  # Red points
            
            # Draw robot position at center
            cv2.circle(lidar_img, (lidar_img_center, lidar_img_center), 8, (255, 0, 0), -1)  # Blue circle for robot
            
            # Add text labels
            cv2.putText(lidar_img, 'Robot', (lidar_img_center + 12, lidar_img_center - 12), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
            cv2.putText(lidar_img, f'Range: {lidar_max_range}m', (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            cv2.putText(lidar_img, f'Points: {len(pointcloud)}', (10, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            
            # Display LIDAR image
            cv2.imshow('2D Lidar Point Cloud', lidar_img)
        
        # Print some info every 60 frames (about 4 times per second at 240Hz)
        if frame_count % 60 == 0:
            print(f"Frame {frame_count}: RGB shape={rgb_img.shape}, Depth range=[{depth_array.min():.3f}, {depth_array.max():.3f}]m, Seg IDs={np.unique(seg_array)}", end='')
            if lidar_link_idx is not None:
                print(f", Lidar points={len(pointcloud)}")
            else:
                print()
        
        frame_count += 1
        
        # Check for 'q' key to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            print("Quitting camera display...")
            break
        
        # Small sleep to prevent overwhe
        # For background/no object pixels (ID < 0 or not in unique_ids), use black
        background_mask = ~np.isin(seg_array, unique_ids)
        seg_colormap[background_mask] = [0, 0, 0]  # Black for background
        
        # Display camera images
        cv2.imshow('RGB Image', rgb_display)
        cv2.imshow('Depth Image', depth_colormap)
        cv2.imshow('Segmented Image', seg_colormap)
        
        # Get and display 2D LIDAR point cloud
        if lidar_link_idx is not None:
            pointcloud = get_2d_lidar_pointcloud(car_id, lidar_link_idx, 
                                                num_rays=lidar_num_rays, 
                                                max_range=lidar_max_range)
        time.sleep(time_step)
        
except KeyboardInterrupt:
    print("Interrupted by user")
finally:
    # Clean up
    cv2.destroyAllWindows()
    print("Camera and LIDAR display windows closed.")
 

Frame 0: RGB shape=(480, 640, 4), Depth range=[0.891, 1.000]m, Seg IDs=[-1  0  3  4  5], Lidar points=360
Frame 60: RGB shape=(480, 640, 4), Depth range=[0.900, 1.000]m, Seg IDs=[-1  0  3  4  5], Lidar points=360
Frame 120: RGB shape=(480, 640, 4), Depth range=[0.900, 1.000]m, Seg IDs=[-1  0  3  4  5], Lidar points=360
Frame 180: RGB shape=(480, 640, 4), Depth range=[0.900, 1.000]m, Seg IDs=[-1  0  3  4  5], Lidar points=360
Frame 240: RGB shape=(480, 640, 4), Depth range=[0.900, 1.000]m, Seg IDs=[-1  0  3  4  5], Lidar points=360
Frame 300: RGB shape=(480, 640, 4), Depth range=[0.900, 1.000]m, Seg IDs=[-1  0  3  4  5], Lidar points=360
Frame 360: RGB shape=(480, 640, 4), Depth range=[0.900, 1.000]m, Seg IDs=[-1  0  3  4  5], Lidar points=360
Frame 420: RGB shape=(480, 640, 4), Depth range=[0.900, 1.000]m, Seg IDs=[-1  0  3  4  5], Lidar points=360
Frame 480: RGB shape=(480, 640, 4), Depth range=[0.900, 1.000]m, Seg IDs=[-1  0  3  4  5], Lidar points=360
Frame 540: RGB shape=(480, 640,